In [1]:
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [2]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2[df2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [3]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]

In [4]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = train_attack[sensors].reset_index().drop(columns=['index'])

In [5]:
hankel = Hankel()
corrhankel = Corrhankel()
lag = 60
stride = 0.5

In [6]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [7]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [8]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [9]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = test_combined.loc[:,'ATT_FLAG']

In [10]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [11]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,
              only_corr=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [12]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.8235294117647058
Precision  0.803921568627451
Recall  0.8913043478260869
F1-score  0.845360824742268


In [14]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.5288645690834475 Median : 0.5176470588235295 Min : 0.4235294117647059 Max : 0.6470588235294118
precision - Mean: 0.7529213272787324 Median : 0.8947368421052632 Min : 0.0 Max : 1.0
recall -    Mean: 0.14408493427704752 Median : 0.15217391304347827 Min : 0.0 Max : 0.3695652173913043
f1 -        Mean: 0.23374893659885193 Median : 0.2545454545454546 Min : 0.0 Max : 0.5230769230769231


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [16]:
fscores_k=[]
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,
                only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.5411764705882353
Precision  0.5411764705882353
Recall  1.0
F1-score  0.7022900763358779
-------number of clusters------ 3
Accuracy  0.5411764705882353
Precision  0.5411764705882353
Recall  1.0
F1-score  0.7022900763358779
-------number of clusters------ 4
Accuracy  0.5411764705882353
Precision  0.5411764705882353
Recall  1.0
F1-score  0.7022900763358779
-------number of clusters------ 5
Accuracy  0.5411764705882353
Precision  0.5411764705882353
Recall  1.0
F1-score  0.7022900763358779
-------number of clusters------ 6
Accuracy  0.5411764705882353
Precision  0.5411764705882353
Recall  1.0
F1-score  0.7022900763358779
-------number of clusters------ 7
Accuracy  0.5411764705882353
Precision  0.5411764705882353
Recall  1.0
F1-score  0.7022900763358779


# Epasad with 1 cluster + No threshold tuning (No concat of training and test data)

In [33]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = df_test.loc[:,'ATT_FLAG']

In [34]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [35]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,
              corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    sensor_models.append(model)    

In [36]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1])

In [37]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.7794117647058824
Precision  0.7058823529411765
Recall  0.8275862068965517
F1-score  0.7619047619047619


In [38]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.6019151846785226 Median : 0.5882352941176471 Min : 0.5147058823529411 Max : 0.7058823529411765
precision - Mean: 0.5523230645323669 Median : 0.6666666666666666 Min : 0.0 Max : 1.0
recall -    Mean: 0.08981555733761026 Median : 0.034482758620689655 Min : 0.0 Max : 0.3103448275862069
f1 -        Mean: 0.1483482023254447 Median : 0.06666666666666667 Min : 0.0 Max : 0.4736842105263158


# Multiple clusters + No threshold tuning (No concat of training and test data)

In [26]:
fscores_k_2=[]
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,
                only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_2.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 3
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 4
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 5
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 6
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 7
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969


# Multiple clusters + Threshold tuning (No concat of training and test data)

In [32]:
fscores_k_3=[]
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=True,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T
                ,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_normal.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print("-------number of clusters------", k)
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted))
    fscores_k_3.append(f1_score(y_actual,y_predicted))

-------number of clusters------ 2
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 3
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 4
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 5
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 6
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
-------number of clusters------ 7
Accuracy  0.4264705882352941
Precision  0.4264705882352941
Recall  1.0
F1-score  0.5979381443298969
